# Criação da Planilha com os Pokemons

Instalação da Dependência

In [244]:
!pip install -q requests gspread google-auth-oauthlib

Imports

In [245]:
import requests
import gspread
from google.oauth2.service_account import Credentials

# --- INÍCIO: BLOCO DE AUTENTICAÇÃO CORRETO ---

# Define o escopo de permissões que nosso script precisa.
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# Carrega as credenciais diretamente do arquivo JSON da Conta de Serviço.
try:
    creds = Credentials.from_service_account_file(
        'credentials.json',
        scopes=scopes
    )
except FileNotFoundError:
    print("ERRO: O arquivo 'credentials.json' não foi encontrado.")
    print("Por favor, faça o upload do arquivo da sua Conta de Serviço para o ambiente do Colab.")
    raise

# Autoriza o gspread com as credenciais carregadas.
gc = gspread.authorize(creds)

Autentica a Planilha

In [246]:
SPREADSHEET_NAME = 'MinhaPokedex'

try:
    # Abre a planilha pelo nome
    spreadsheet = gc.open(SPREADSHEET_NAME)
    print(f"Planilha '{SPREADSHEET_NAME}' acessada com sucesso!")
except gspread.exceptions.SpreadsheetNotFound:
    print(f"ERRO: A planilha '{SPREADSHEET_NAME}' não foi encontrada ou não foi compartilhada.")
    print(f"Verifique se o nome está correto e se você compartilhou a planilha com o e-mail do seu arquivo 'credentials.json' como 'Editor'.")
    raise

Planilha 'MinhaPokedex' acessada com sucesso!


Coleta o Tipo do Pokemon

In [247]:
def get_pokemon_types(pokemon_name: str) -> tuple[list, str | None]:
    """
    Busca os tipos e a URL da sprite frontal padrão de um Pokémon na PokeAPI.
    Retorna uma tupla contendo uma lista de tipos e a URL da sprite (ou None se o Pokémon não for encontrado ou a sprite não estiver disponível).
    """
    pokemon_name = pokemon_name.lower().strip()
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_name}"
    print(f"Buscando dados para '{pokemon_name}' na PokeAPI...")

    try:
        response = requests.get(url)
        # Lança um erro para respostas ruins (ex: 404 Not Found)
        response.raise_for_status()

        data = response.json()
        types = [t['type']['name'] for t in data['types']]
        print(f"Tipos encontrados: {types}")

        # Acessa a URL da sprite frontal padrão
        sprite_url = data.get('sprites', {}).get('front_default')
        if sprite_url:
            print(f"URL da sprite encontrada: {sprite_url}")
        else:
            print("URL da sprite não encontrada.")

        return types, sprite_url

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Pokémon '{pokemon_name}' não encontrado.")
        else:
            print(f"Erro na requisição à API: {e}")
        return None, None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None, None

Gera os Pokemons

In [248]:
import requests
import random

# Lista de nomes de Pokémon para buscar dados
# Adicione ou remova nomes de Pokémon nesta lista

# Função para buscar uma lista de Pokémon de uma determinada URL da PokeAPI
def get_pokemon_list(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return [item['pokemon_species']['name'] for item in data['pokemon_entries']]
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar lista de Pokémon da URL {url}: {e}")
        return []

# URLs da PokeAPI para diferentes gerações ou regiões que cubram as 9 gerações
# Você pode ajustar estas URLs para incluir mais Pokémon ou gerações específicas se desejar.
# Exemplo: Geração 1 (Kanto), Geração 2 (Johto), etc., ou regiões que englobam várias gerações.
# Vamos tentar buscar Pokémon de algumas regiões que juntas cubram as 9 gerações principais.
# Note: A PokeAPI pode ter diferentes estruturas ou endpoints para gerações/regiões.
# Vamos usar um endpoint que lista espécies por região para tentar cobrir mais ground.
# Region URLs can give a list of locations, not directly species by generation.
# Let's try Pokedex endpoints which are tied to regions/games and often list species.
# URLs de Pokedex que cobrem as 9 gerações (pode precisar ajustar dependendo da versão da API ou cobertura desejada)
pokedex_urls = [
    "https://pokeapi.co/api/v2/pokedex/kanto/", # Geração 1
    "https://pokeapi.co/api/v2/pokedex/johto/", # Geração 2
    "https://pokeapi.co/api/v2/pokedex/hoenn/", # Geração 3
    "https://pokeapi.co/api/v2/pokedex/sinnoh/", # Geração 4
    "https://pokeapi.co/api/v2/pokedex/unova/", # Geração 5
    "https://pokeapi.co/api/v2/pokedex/kalos/", # Geração 6 (Central Kalos)
    "https://pokeapi.co/api/v2/pokedex/alola/", # Geração 7 (Melemele)
    "https://pokeapi.co/api/v2/pokedex/galar/", # Geração 8 (Isle of Armor)
    "https://pokeapi.co/api/v2/pokedex/paldea/", # Geração 9 (Paldea)
    # Adicione mais URLs de Pokedex/Região se necessário para cobrir todos os 1000+ Pokémon
    "https://pokeapi.co/api/v2/pokedex/hisui/", # Legends Arceus (mistura de gerações)
    "https://pokeapi.co/api/v2/pokedex/kitakami/", # Scarlet/Violet DLC
    "https://pokeapi.co/api/v2/pokedex/blueberry/", # Scarlet/Violet DLC
]

all_pokemon_names = set() # Usar um conjunto para evitar duplicatas

print("Buscando listas de Pokémon das Pokedex/Regiões...")
for url in pokedex_urls:
    print(f"Buscando em: {url}")
    pokemon_list_from_url = get_pokemon_list(url)
    if pokemon_list_from_url:
        all_pokemon_names.update(pokemon_list_from_url)

# Converte o conjunto de volta para lista
all_pokemon_names = list(all_pokemon_names)

print(f"\nTotal de nomes de Pokémon únicos encontrados: {len(all_pokemon_names)}")

# Seleciona 20 nomes aleatórios se houver Pokémon suficientes
num_to_select = 20
if len(all_pokemon_names) >= num_to_select:
    pokemons_to_process = random.sample(all_pokemon_names, num_to_select)
    print(f"\nSelecionados {num_to_select} Pokémon aleatórios:")
    print(pokemons_to_process)
else:
    print(f"\nAviso: Não há Pokémon suficientes ({len(all_pokemon_names)}) para selecionar {num_to_select} aleatórios.")
    print("Usando todos os Pokémon encontrados.")
    pokemons_to_process = all_pokemon_names

# Você ainda pode adicionar nomes manualmente à lista pokemons_to_process após esta seleção aleatória, se desejar.
# Exemplo:
# pokemons_to_process.append('charmander')
# pokemons_to_process.append('venusaur')

Buscando listas de Pokémon das Pokedex/Regiões...
Buscando em: https://pokeapi.co/api/v2/pokedex/kanto/
Buscando em: https://pokeapi.co/api/v2/pokedex/johto/
Erro ao buscar lista de Pokémon da URL https://pokeapi.co/api/v2/pokedex/johto/: 404 Client Error: Not Found for url: https://pokeapi.co/api/v2/pokedex/johto/
Buscando em: https://pokeapi.co/api/v2/pokedex/hoenn/
Buscando em: https://pokeapi.co/api/v2/pokedex/sinnoh/
Erro ao buscar lista de Pokémon da URL https://pokeapi.co/api/v2/pokedex/sinnoh/: 404 Client Error: Not Found for url: https://pokeapi.co/api/v2/pokedex/sinnoh/
Buscando em: https://pokeapi.co/api/v2/pokedex/unova/
Erro ao buscar lista de Pokémon da URL https://pokeapi.co/api/v2/pokedex/unova/: 404 Client Error: Not Found for url: https://pokeapi.co/api/v2/pokedex/unova/
Buscando em: https://pokeapi.co/api/v2/pokedex/kalos/
Erro ao buscar lista de Pokémon da URL https://pokeapi.co/api/v2/pokedex/kalos/: 404 Client Error: Not Found for url: https://pokeapi.co/api/v2/po

Armazena os dados por Tipo

In [249]:
# Inicializa um dicionário vazio para armazenar dados organizados por tipo
pokemon_by_type = {}

# Inicializa uma lista vazia para armazenar Pokémon sem tipo
untied_pokemon = []

# Itera pela lista de nomes de Pokémon
for pokemon in pokemons_to_process:
    # Recupera tipos e URL da sprite usando a função existente
    types, sprite_url = get_pokemon_types(pokemon)

    # Verifica se os tipos foram recuperados com sucesso
    if types is not None:
        # Se o Pokémon tiver tipos
        if types:
            for pokemon_type in types:
                # Adiciona o tipo ao dicionário se ele não existir
                if pokemon_type not in pokemon_by_type:
                    pokemon_by_type[pokemon_type] = []

                # Adiciona o nome do Pokémon (capitalizado) e a URL da sprite à lista para este tipo
                pokemon_by_type[pokemon_type].append((pokemon.capitalize(), sprite_url))
        # Se o Pokémon não tiver tipo atribuído (lista de tipos vazia)
        else:
            untied_pokemon.append((pokemon.capitalize(), sprite_url))
    else:
        # Lida com casos em que get_pokemon_types falhou (ex: Pokémon não encontrado)
        print(f"Não foi possível recuperar tipos para {pokemon.capitalize()}. Pulando.")

# Imprime os dados organizados e a lista de Pokémon sem tipo
print("\nDados de Pokémon organizados por tipo:")
print(pokemon_by_type)

print("\nPokémon sem tipo:")
print(untied_pokemon)

Buscando dados para 'torracat' na PokeAPI...
Tipos encontrados: ['fire']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png
Buscando dados para 'applin' na PokeAPI...
Tipos encontrados: ['grass', 'dragon']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/840.png
Buscando dados para 'tynamo' na PokeAPI...
Tipos encontrados: ['electric']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/602.png
Buscando dados para 'sinistea' na PokeAPI...
Tipos encontrados: ['ghost']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/854.png
Buscando dados para 'dragalge' na PokeAPI...
Tipos encontrados: ['poison', 'dragon']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/691.png
Buscando dados para 'sandygast' na PokeAPI...
Tipos encontrados: ['ghost', '

Cria a Lista

In [250]:
# Inicializa uma lista vazia chamada rows_to_write
rows_to_write = []

# Itera através do dicionário pokemon_by_type
for pokemon_type, pokemon_list in pokemon_by_type.items():
    # Para cada tupla de Pokémon (nome, sprite_url) na lista
    for pokemon_name, sprite_url in pokemon_list:
        # Cria uma lista representando uma linha na planilha
        # Formata a URL da sprite como uma fórmula IMAGE() do Google Sheets
        sprite_formula = f'=IMAGE("{sprite_url}")' if sprite_url else "N/A"
        row = [pokemon_type.capitalize(), pokemon_name, sprite_formula]

        # Adiciona esta lista de linha ao rows_to_write
        rows_to_write.append(row)

# Se a lista untied_pokemon não estiver vazia, itera através dela (atualmente vazia, mas incluída para completude)
for pokemon_name, sprite_url in untied_pokemon:
    # Cria uma lista de linha para pokemon sem tipo
    sprite_formula = f'=IMAGE("{sprite_url}")' if sprite_url else "N/A"
    row = ["Untied", pokemon_name, sprite_formula]
    # Adiciona esta lista de linha ao rows_to_write
    rows_to_write.append(row)

# Imprime a lista rows_to_write para verificar a estrutura dos dados
print("Rows to write to the spreadsheet:")
display(rows_to_write)

Rows to write to the spreadsheet:


[['Fire',
  'Torracat',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png")'],
 ['Fire',
  'Delphox',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/655.png")'],
 ['Fire',
  'Emboar',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/500.png")'],
 ['Fire',
  'Ninetales',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/38.png")'],
 ['Grass',
  'Applin',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/840.png")'],
 ['Grass',
  'Tangela',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/114.png")'],
 ['Grass',
  'Serperior',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/497.png")'],
 ['Grass',
  'Sawsbuck',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/586.png")'],
 ['Dragon',
  'Applin',
  '=IMAGE

Organiza a Planilha

In [251]:
def write_organized_data_to_sheet(rows_to_write: list, pokemon_type_colors: dict):
    """
    Escreve os dados organizados de Pokémon na aba 'Pokedex', limpando conteúdo anterior
    (mantendo cabeçalho), e aplicando cores de fundo baseadas no tipo e fórmulas de imagem.
    """
    MAIN_SHEET_NAME = 'Pokedex'
    HEADER = ["Tipo", "Nome do Pokemon", "Sprite"]

    print(f"Escrevendo dados organizados na planilha '{MAIN_SHEET_NAME}'...")

    try:
        # Obtém a aba principal pelo nome
        main_worksheet = spreadsheet.worksheet(MAIN_SHEET_NAME)
        print(f"Aba '{MAIN_SHEET_NAME}' acessada com sucesso.")
    except gspread.exceptions.WorksheetNotFound:
        # Se não encontrada, cria a aba com o cabeçalho
        print(f"Aba '{MAIN_SHEET_NAME}' não encontrada. Criando...")
        # Ajusta cols conforme necessário, garantindo colunas suficientes para o HEADER
        main_worksheet = spreadsheet.add_worksheet(title=MAIN_SHEET_NAME, rows="1000", cols=str(len(HEADER)))
        # Adiciona a linha do cabeçalho
        main_worksheet.append_row(HEADER)
        print(f"Aba '{MAIN_SHEET_NAME}' criada com cabeçalho.")

    # Limpa o conteúdo existente, mantendo a linha do cabeçalho
    # Obtém o número total de linhas atualmente na planilha
    all_values = main_worksheet.get_all_values()
    if len(all_values) > 1: # Verifica se há linhas além do cabeçalho
        num_existing_rows = len(all_values)
        # Define o intervalo a ser limpo (da segunda linha até a última linha)
        range_to_clear = f'A2:{chr(ord("A") + len(HEADER) - 1)}{num_existing_rows}'
        print(f"Limpando intervalo: {range_to_clear}")
        try:
            main_worksheet.batch_clear([range_to_clear])
            print("Conteúdo existente limpo (mantendo cabeçalho).")
        except Exception as e:
            print(f"Erro ao limpar conteúdo da planilha: {e}")


    # Escreve os dados preparados (rows_to_write) na planilha
    if rows_to_write:
        try:
            # Escreve as linhas começando da segunda linha (após o cabeçalho)
            main_worksheet.append_rows(rows_to_write, value_input_option='USER_ENTERED') # USER_ENTERED para processar fórmulas
            print(f"{len(rows_to_write)} linhas de dados escritas em '{MAIN_SHEET_NAME}'.")
        except Exception as e:
            print(f"Erro ao escrever dados na planilha: {e}")

        # Aplica cor de fundo à célula "Tipo" para cada linha
        # Começa da segunda linha (índice 1 em uma lista baseada em 0)
        start_row_index = 2 # O número da linha no Google Sheets começa em 1, os dados começam na linha 2
        for i, row_data in enumerate(rows_to_write):
            # O tipo está na primeira coluna (índice 0) dos dados da linha
            pokemon_type = row_data[0].lower()
            color_code = pokemon_type_colors.get(pokemon_type)

            if color_code:
                # O endereço da célula está na coluna A (índice 0) e no número da linha atual
                cell_address = f'A{start_row_index + i}'
                # print(f"Attempting to format cell {cell_address} with color {color_code}") # Debug print
                try:
                    main_worksheet.format(cell_address, {
                        "backgroundColor": {
                            "red": int(color_code[1:3], 16) / 255,
                            "green": int(color_code[3:5], 16) / 255,
                            "blue": int(color_code[5:7], 16) / 255
                        }
                    })
                    # print(f"Formatted cell {cell_address} successfully.") # Debug print
                except Exception as e:
                    print(f"Erro ao formatar a célula {cell_address} para o tipo '{pokemon_type}': {e}")
            # else:
                # print(f"No color found for type '{pokemon_type}'. Skipping formatting for cell A{start_row_index + i}.") # Debug print


    else:
        print("Nenhuma linha para escrever.")

    print("Escrita de dados organizados finalizada.")

Remove o Loop

In [252]:
# Remove o loop anterior que chamava run_pokemon_agent para cada Pokémon individualmente.
# O bloco de código anterior para o loop era:
# for pokemon in pokemons_to_process:
#     run_pokemon_agent(pokemon)

# A lógica para obter e organizar dados já está nas saídas de execução das células anteriores.
# Temos o dicionário 'pokemon_by_type' e a lista 'rows_to_write' dos passos anteriores.

# Agora, chama a função write_organized_data_to_sheet com os dados preparados.
# O dicionário 'pokemon_type_colors' já está disponível nas células anteriores.
write_organized_data_to_sheet(rows_to_write, pokemon_type_colors)

Escrevendo dados organizados na planilha 'Pokedex'...
Aba 'Pokedex' acessada com sucesso.
Limpando intervalo: A2:C29
Conteúdo existente limpo (mantendo cabeçalho).
26 linhas de dados escritas em 'Pokedex'.
Escrita de dados organizados finalizada.


# Agente IA

Instalar Dependência

In [253]:
!pip install -q google-generativeai

Import da Chave

In [254]:
import google.generativeai as genai
from google.colab import userdata

# Carrega a chave de API do Colab Secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except userdata.SecretNotFoundError:
    print("ERRO: Chave de API 'GOOGLE_API_KEY' não encontrada no Colab Secrets.")
    print("Por favor, adicione sua chave de API no painel 'Segredos' (ícone de chave 🔑).")
    GOOGLE_API_KEY = None # Define como None para evitar configurar com valor inválido

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API do Google Generative AI configurada com sucesso.")
else:
    print("Não foi possível configurar a API devido à falta da chave.")

API do Google Generative AI configurada com sucesso.


Inicializar o Modelo

In [255]:
if GOOGLE_API_KEY:
    try:
        # Inicializa o modelo
        gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')
        print("Modelo Gemini inicializado com sucesso.")
    except Exception as e:
        print(f"Erro ao inicializar o modelo Gemini: {e}")
        gemini_model = None
else:
    print("Não foi possível inicializar o modelo sem a chave de API.")
    gemini_model = None

Modelo Gemini inicializado com sucesso.


# Criação da Equipe
Usando a lista de 20 Pokemons, a IA irá criar um time com os melhores pokemons selecionados.

Obter a lista `pokemons_to_process` e obter os tipos para cada um desses Pokémon.


In [256]:
# 1. Cria uma nova lista vazia chamada pokemon_list_with_types.
pokemon_list_with_types = []

# 2. Itera sobre cada nome de Pokémon na lista pokemons_to_process.
for pokemon in pokemons_to_process:
    # 3. Para cada Pokémon, chama a função get_pokemon_types para obter sua lista de tipos e a URL da sprite.
    types, sprite_url = get_pokemon_types(pokemon)

    # 4. Se a função retornar tipos (ou seja, não for None), adiciona uma tupla contendo o nome do Pokémon (capitalizado) e a lista de seus tipos à lista pokemon_list_with_types.
    if types is not None:
        pokemon_list_with_types.append((pokemon.capitalize(), types))

# 5. Imprime a lista pokemon_list_with_types para verificar os dados coletados.
print("Lista de Pokémon com tipos:")
display(pokemon_list_with_types)

Buscando dados para 'torracat' na PokeAPI...
Tipos encontrados: ['fire']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png
Buscando dados para 'applin' na PokeAPI...
Tipos encontrados: ['grass', 'dragon']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/840.png
Buscando dados para 'tynamo' na PokeAPI...
Tipos encontrados: ['electric']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/602.png
Buscando dados para 'sinistea' na PokeAPI...
Tipos encontrados: ['ghost']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/854.png
Buscando dados para 'dragalge' na PokeAPI...
Tipos encontrados: ['poison', 'dragon']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/691.png
Buscando dados para 'sandygast' na PokeAPI...
Tipos encontrados: ['ghost', '

[('Torracat', ['fire']),
 ('Applin', ['grass', 'dragon']),
 ('Tynamo', ['electric']),
 ('Sinistea', ['ghost']),
 ('Dragalge', ['poison', 'dragon']),
 ('Sandygast', ['ghost', 'ground']),
 ('Delphox', ['fire', 'psychic']),
 ('Palpitoad', ['water', 'ground']),
 ('Tangela', ['grass']),
 ('Primeape', ['fighting']),
 ('Joltik', ['bug', 'electric']),
 ('Emboar', ['fire', 'fighting']),
 ('Hatenna', ['psychic']),
 ('Exploud', ['normal']),
 ('Ninetales', ['fire']),
 ('Serperior', ['grass']),
 ('Sawsbuck', ['normal', 'grass']),
 ('Mudkip', ['water'])]

Define o Time

In [257]:
# 1. Define a variável team_size.
team_size = 6

# 2. Constrói a string do prompt para o modelo de IA.
prompt_for_team = f"""
Você é um especialista em Pokémon e em construção de equipes equilibradas.
Aqui está uma lista de Pokémon disponíveis e seus tipos:

"""

# Adiciona a lista de Pokémon e tipos ao prompt
for pokemon_name, pokemon_types in pokemon_list_with_types:
    types_str = ", ".join([t.capitalize() for t in pokemon_types])
    prompt_for_team += f"- {pokemon_name}: {types_str}\n"

prompt_for_team += f"""

Com base nesta lista, por favor, sugira o melhor time possível de {team_size} Pokémon.
A sugestão deve levar em consideração um bom equilíbrio de tipos para cobrir diversas fraquezas e ter forças contra diferentes tipos.
Por favor, explique brevemente o raciocínio por trás da escolha de cada Pokémon na equipe sugerida e como o time funciona em conjunto.

Responda em português.
"""

# 3. Armazena a string do prompt na variável prompt_for_team (já feito acima).

# 4. Imprime a variável prompt_for_team para revisar o prompt gerado.
print(prompt_for_team)


Você é um especialista em Pokémon e em construção de equipes equilibradas.
Aqui está uma lista de Pokémon disponíveis e seus tipos:

- Torracat: Fire
- Applin: Grass, Dragon
- Tynamo: Electric
- Sinistea: Ghost
- Dragalge: Poison, Dragon
- Sandygast: Ghost, Ground
- Delphox: Fire, Psychic
- Palpitoad: Water, Ground
- Tangela: Grass
- Primeape: Fighting
- Joltik: Bug, Electric
- Emboar: Fire, Fighting
- Hatenna: Psychic
- Exploud: Normal
- Ninetales: Fire
- Serperior: Grass
- Sawsbuck: Normal, Grass
- Mudkip: Water


Com base nesta lista, por favor, sugira o melhor time possível de 6 Pokémon.
A sugestão deve levar em consideração um bom equilíbrio de tipos para cobrir diversas fraquezas e ter forças contra diferentes tipos.
Por favor, explique brevemente o raciocínio por trás da escolha de cada Pokémon na equipe sugerida e como o time funciona em conjunto.

Responda em português.



Verifica se modelo está disponível

In [258]:
# 1. Verifique se a variável gemini_model não é None.
if gemini_model:
    print("Modelo Gemini disponível. Gerando sugestão de equipe...")
    try:
        # 2. Use o método generate_content() do gemini_model, passando a variável prompt_for_team como argumento.
        team_suggestion_response = gemini_model.generate_content(prompt_for_team)

        # 3. Armazene a resposta do modelo em uma variável (team_suggestion_response).
        # (Já feito na linha acima)

        print("Sugestão de equipe gerada com sucesso.")

    # 4. Inclua um bloco try...except para capturar possíveis erros.
    except Exception as e:
        print(f"Erro ao gerar a sugestão de equipe com o modelo Gemini: {e}")
        team_suggestion_response = None # Define como None em caso de erro
else:
    print("Modelo Gemini não disponível. Não foi possível gerar a sugestão de equipe.")
    team_suggestion_response = None

Modelo Gemini disponível. Gerando sugestão de equipe...
Sugestão de equipe gerada com sucesso.


Sugere o time

In [259]:
# Verifica se a resposta do modelo foi obtida com sucesso
if team_suggestion_response and team_suggestion_response.text:
    # Exibe a sugestão do agente de IA em uma nova célula Markdown.
    # Usa display(Markdown(...)) para renderizar o texto como Markdown.
    from IPython.display import display, Markdown
    display(Markdown("## Sugestão de Equipe do Agente de IA\n\n" + team_suggestion_response.text))
elif team_suggestion_response:
    # Se a resposta não tiver texto, mas o objeto de resposta existir
    print("O modelo gerou uma resposta, mas ela não contém texto visível.")
else:
    # Se team_suggestion_response for None (em caso de erro anterior)
    print("Não foi possível exibir a sugestão da equipe, pois a resposta do modelo não foi obtida.")

## Sugestão de Equipe do Agente de IA

A construção de uma equipe Pokémon eficiente requer uma boa cobertura de tipos para minimizar as fraquezas.  Baseado na sua lista, aqui está uma sugestão de time de 6 Pokémon e a justificativa por trás da escolha:

**Equipe Sugerida:**

1. **Torracat (Fogo):** Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.
2. **Dragalge (Veneno/Dragão):** Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.
3. **Palpitoad (Água/Terra):**  Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.
4. **Sinistea (Fantasma):**  O tipo fantasma é imune a ataques normais e luta, oferecendo uma camada importante de defesa.  É efetivo contra fantasmas, psicicos e outros tipos, e cobre as fraquezas dos outros.
5. **Primeape (Luta):** Oferece cobertura para tipos de rocha, aço, gelo e normal que os outros Pokémon do time não abordam tão bem.
6. **Joltik (Inseto/Elétrico):** Apesar de Inseto ser um tipo com muitas fraquezas, a combinação com Elétrico oferece cobertura contra Água, Voador e outros.  Além disso, Joltik pode ser um bom setup sweeper com algumas estratégias.


**Raciocínio:**

Este time tenta cobrir a maioria dos tipos com eficácia.  A escolha foi direcionada para maximizar a sinergia entre os tipos, minimizando as fraquezas coletivas.  Temos boa cobertura contra:

* **Fogo:** Palpitoad (Água) e Dragalge (Dragão)
* **Água:**  Primeape (Luta) e Joltik (Elétrico)
* **Planta:** Torracat (Fogo) e Dragalge (Veneno/Dragão)
* **Elétrico:** Palpitoad (Terra)
* **Grama:** Torracat (Fogo)
* **Gelo:** Torracat (Fogo) e Primeape (Luta)
* **Terra:** Joltik (Elétrico),  e Dragalge (Dragão)
* **Pedra:** Primeape (Luta)
* **Aço:** Torracat (Fogo) e Primeape (Luta)


**Pontos Fracos da Equipe:**

A equipe ainda tem algumas fraquezas residuais, principalmente contra Psíquico (embora Sinistea ajude aqui), mas a construção visa a melhor cobertura possível com os Pokémon disponíveis.  A escolha de um Pokémon tipo Psíquico seria ideal, mas a ausência de bons candidatos na lista original limita um pouco a formação.

**Observação:**  Esta sugestão presume um cenário de batalha equilibrada.  A escolha de movimentos individuais para cada Pokémon seria crucial para otimizar o desempenho da equipe em diferentes situações.


# Pagina de Time
Crie uma segunda página na Planilhas com o nome "Time" Trazendo a sugestão, com o nome, tipo sprite e estratégia dos 6 pokemon. Fazendo ela um pouco mais decorada e colorida

In [260]:
import re

# 1. Verifique se team_suggestion_response não é None e se team_suggestion_response.text contém texto.
if team_suggestion_response and team_suggestion_response.text:
    # 2. Divida a string team_suggestion_response.text em linhas.
    lines = team_suggestion_response.text.strip().splitlines()

    suggested_team_details = []
    current_pokemon_name = None
    current_strategy = []
    capturing_strategy = False

    # 3. Itera sobre as linhas e identifique as linhas que contêm as sugestões individuais de Pokémon.
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Verifica se a linha começa com um número seguido por um ponto e espaço, indicando um novo Pokémon.
        match = re.match(r'^(\d+)\.\s*\*\*(.*?)\*\*', line)
        if match:
            # Se encontrarmos um novo Pokémon, armazena os detalhes do Pokémon anterior (se houver)
            if current_pokemon_name:
                suggested_team_details.append({
                    "Nome do Pokemon": current_pokemon_name,
                    "Estratégia": " ".join(current_strategy).strip()
                })
            # Extrai APENAS o nome do Pokémon, removendo o tipo entre parênteses
            full_name = match.group(2).strip()
            name_match = re.match(r'(.*?)\s*\(.*?\)', full_name)
            if name_match:
                current_pokemon_name = name_match.group(1).strip()
            else:
                current_pokemon_name = full_name
                # Fallback se não houver tipo entre parênteses

            current_strategy = []
            capturing_strategy = True
            # Começa a capturar as linhas de estratégia

            # Adiciona o restante da linha após o nome como o início da estratégia
            remaining_line = line[match.end():].strip()
            if remaining_line:
                 # Remove as informações de tipo entre parênteses e o caractere de dois pontos, se presentes
                type_match = re.search(r'\s*\(.*?\):', remaining_line)
                if type_match:
                    remaining_line = remaining_line[type_match.end():].strip()
                if remaining_line:
                     current_strategy.append(remaining_line)

        elif capturing_strategy:
            # Se estivermos capturando a estratégia e a linha não indicar um novo Pokémon, adicione a linha à estratégia atual.
            # Para de capturar a estratégia se encontrarmos uma linha que parece uma nova seção ou resumo
            if line.lower().startswith("raciocínio:") or line.lower().startswith("considerando"):
                 capturing_strategy = False
                 # Podemos ter capturado o início da próxima seção como estratégia, então removemos isso de current_strategy
                 if current_strategy and (current_strategy[-1].lower().startswith("raciocínio:") or current_strategy[-1].lower().startswith("considerando")):
                    current_strategy.pop()
            else:
                current_strategy.append(line)

    # Adiciona o último Pokémon e sua estratégia após o loop
    if current_pokemon_name:
        suggested_team_details.append({
            "Nome do Pokemon": current_pokemon_name,
            "Estratégia": " ".join(current_strategy).strip()
        })

    # 5. Armazene os nomes dos Pokémon e suas estratégias extraídas em uma lista de dicionários.
    print("Detalhes da equipe sugerida extraídos:")
    display(suggested_team_details)

else:
    print("Não foi possível extrair os detalhes da equipe. A resposta do modelo estava vazia ou None.")

Detalhes da equipe sugerida extraídos:


[{'Nome do Pokemon': 'Torracat',
  'Estratégia': 'Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.'},
 {'Nome do Pokemon': 'Dragalge',
  'Estratégia': 'Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.'},
 {'Nome do Pokemon': 'Palpitoad',
  'Estratégia': 'Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.'},
 {'Nome do Pokemon': 'Sinistea',
  'Estratégia': 'O tipo fantasma é imune a ataques normais e luta, oferecendo uma camada importante de defesa.  É efetivo contra fantasmas, psicicos e outros tipos, e cobre as fraquezas dos outros.'},
 {'Nome do Pokemon': 'Primeape',
  'Estratégia': 'Oferece cobertura para tipos de rocha, aço, gelo e normal que os outros Pokémon do time não abordam tão bem.'},
 {'Nome do Pokemon': 'Joltik',
  'Estratégia'

Limpa a Estratégia

In [261]:
# Limpa a última entrada de estratégia para remover a parte "Raciocínio".
if suggested_team_details:
    last_entry = suggested_team_details[-1]
    strategy = last_entry["Estratégia"]
    # Encontra o início da seção "Raciocínio" e trunca a estratégia ali.
    raciocinio_index = strategy.lower().find("raciocínio:")
    if raciocinio_index != -1:
        last_entry["Estratégia"] = strategy[:raciocinio_index].strip()
    # Também verifica a parte "Considerando" no início caso a parte "Raciocínio" não tenha sido encontrada
    elif strategy.lower().startswith("considerando"):
         last_entry["Estratégia"] = "" # Or handle as needed if the whole strategy is just the summary intro
         # Ou lida conforme necessário se toda a estratégia for apenas a introdução do resumo


# Exibe a lista de dicionários limpa
print("Detalhes finais da equipe sugerida extraídos:")
display(suggested_team_details)

# Remove o "**" final da última entrada de estratégia.
if suggested_team_details:
    last_entry = suggested_team_details[-1]
    strategy = last_entry["Estratégia"]
    if strategy.endswith("**"):
        last_entry["Estratégia"] = strategy[:-2].strip()

# Exibe a lista final limpa de dicionários
print("Detalhes finais da equipe sugerida extraídos:")
display(suggested_team_details)

Detalhes finais da equipe sugerida extraídos:


[{'Nome do Pokemon': 'Torracat',
  'Estratégia': 'Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.'},
 {'Nome do Pokemon': 'Dragalge',
  'Estratégia': 'Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.'},
 {'Nome do Pokemon': 'Palpitoad',
  'Estratégia': 'Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.'},
 {'Nome do Pokemon': 'Sinistea',
  'Estratégia': 'O tipo fantasma é imune a ataques normais e luta, oferecendo uma camada importante de defesa.  É efetivo contra fantasmas, psicicos e outros tipos, e cobre as fraquezas dos outros.'},
 {'Nome do Pokemon': 'Primeape',
  'Estratégia': 'Oferece cobertura para tipos de rocha, aço, gelo e normal que os outros Pokémon do time não abordam tão bem.'},
 {'Nome do Pokemon': 'Joltik',
  'Estratégia'

Detalhes finais da equipe sugerida extraídos:


[{'Nome do Pokemon': 'Torracat',
  'Estratégia': 'Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.'},
 {'Nome do Pokemon': 'Dragalge',
  'Estratégia': 'Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.'},
 {'Nome do Pokemon': 'Palpitoad',
  'Estratégia': 'Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.'},
 {'Nome do Pokemon': 'Sinistea',
  'Estratégia': 'O tipo fantasma é imune a ataques normais e luta, oferecendo uma camada importante de defesa.  É efetivo contra fantasmas, psicicos e outros tipos, e cobre as fraquezas dos outros.'},
 {'Nome do Pokemon': 'Primeape',
  'Estratégia': 'Oferece cobertura para tipos de rocha, aço, gelo e normal que os outros Pokémon do time não abordam tão bem.'},
 {'Nome do Pokemon': 'Joltik',
  'Estratégia'

Cria a Lista dos Dados dos Pokemons

In [262]:
# 1. Crie uma nova lista vazia chamada team_pokemon_data.
team_pokemon_data = []

# 2. Itera sobre cada dicionário na lista suggested_team_details.
for pokemon_detail in suggested_team_details:
    # 3. Para cada dicionário, obtenha o nome do Pokémon da chave "Nome do Pokemon".
    pokemon_name = pokemon_detail.get("Nome do Pokemon")
    strategy = pokemon_detail.get("Estratégia", "")

    if pokemon_name:
        # 4. Chame a função get_pokemon_types() com o nome do Pokémon para obter seus tipos e a URL da sprite.
        types, sprite_url = get_pokemon_types(pokemon_name)

        # 5. Se a função retornar tipos (ou seja, não for None), crie um novo dicionário para este Pokémon.
        if types is not None:
            pokemon_data = {
                "Nome do Pokemon": pokemon_name,
                "Tipos": types,
                "Sprite URL": sprite_url,
                "Estratégia": strategy
            }
            # 6. Adicione o novo dicionário à lista team_pokemon_data.
            team_pokemon_data.append(pokemon_data)
        else:
            print(f"Não foi possível obter tipos e sprite para '{pokemon_name}'. Pulando.")
    else:
        print("Nome do Pokémon não encontrado no detalhe. Pulando.")


# 7. Imprima a lista team_pokemon_data para verificar os dados coletados.
print("\nDados da equipe de Pokémon com tipos e URLs de sprite:")
display(team_pokemon_data)

Buscando dados para 'torracat' na PokeAPI...
Tipos encontrados: ['fire']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png
Buscando dados para 'dragalge' na PokeAPI...
Tipos encontrados: ['poison', 'dragon']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/691.png
Buscando dados para 'palpitoad' na PokeAPI...
Tipos encontrados: ['water', 'ground']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/536.png
Buscando dados para 'sinistea' na PokeAPI...
Tipos encontrados: ['ghost']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/854.png
Buscando dados para 'primeape' na PokeAPI...
Tipos encontrados: ['fighting']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/57.png
Buscando dados para 'joltik' na PokeAPI...
Tipos encontrados: ['bug', 'e

[{'Nome do Pokemon': 'Torracat',
  'Tipos': ['fire'],
  'Sprite URL': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png',
  'Estratégia': 'Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.'},
 {'Nome do Pokemon': 'Dragalge',
  'Tipos': ['poison', 'dragon'],
  'Sprite URL': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/691.png',
  'Estratégia': 'Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.'},
 {'Nome do Pokemon': 'Palpitoad',
  'Tipos': ['water', 'ground'],
  'Sprite URL': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/536.png',
  'Estratégia': 'Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.'},
 {'Nome do Pokemon': 'Sinistea',
  'Tipos': ['ghost'],


Cria a Lista para mostrar os Pokemons

In [263]:
# 1. Crie uma nova lista vazia chamada team_rows_to_write.
team_rows_to_write = []

# 2. Itera sobre cada dicionário na lista team_pokemon_data.
for pokemon_data in team_pokemon_data:
    # 3. Para cada dicionário, extraia o "Nome do Pokemon", a lista de "Tipos", a "Sprite URL" e a "Estratégia".
    pokemon_name = pokemon_data.get("Nome do Pokemon", "N/A")
    pokemon_types = pokemon_data.get("Tipos", [])
    sprite_url = pokemon_data.get("Sprite URL")
    strategy = pokemon_data.get("Estratégia", "N/A")

    # 4. Converta a lista de "Tipos" em uma única string separada por vírgulas.
    types_str = ", ".join([t.capitalize() for t in pokemon_types]) if pokemon_types else "N/A"

    # 5. Formate a "Sprite URL" como uma fórmula =IMAGE("URL_DA_SPRITE"). Se a URL for None ou vazia, use "N/A".
    sprite_formula = f'=IMAGE("{sprite_url}")' if sprite_url else "N/A"

    # 6. Crie uma lista com os elementos na ordem desejada para a linha da planilha: [Nome do Pokemon, String de Tipos, Fórmula da Sprite, Estratégia].
    row = [pokemon_name, types_str, sprite_formula, strategy]

    # 7. Adicione esta lista à lista team_rows_to_write.
    team_rows_to_write.append(row)

# 8. Imprima a lista team_rows_to_write para verificar a estrutura dos dados.
print("Rows to write to the 'Time' sheet:")
display(team_rows_to_write)

Rows to write to the 'Time' sheet:


[['Torracat',
  'Fire',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png")',
  'Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.'],
 ['Dragalge',
  'Poison, Dragon',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/691.png")',
  'Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.'],
 ['Palpitoad',
  'Water, Ground',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/536.png")',
  'Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.'],
 ['Sinistea',
  'Ghost',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/854.png")',
  'O tipo fantasma é imune a ataques normais e luta, oferecendo uma camada importa

Formata a aba Time

In [264]:
# 1. Define o nome da aba como "Time" e crie a lista TEAM_HEADER.
TEAM_SHEET_NAME = 'Time'
TEAM_HEADER = ["Nome", "Tipo(s)", "Sprite", "Estratégia Sugerida"]

print(f"Preparando para escrever dados na planilha '{TEAM_SHEET_NAME}'...")

try:
    # 2. Tenta obter a aba "Time".
    team_worksheet = spreadsheet.worksheet(TEAM_SHEET_NAME)
    print(f"Aba '{TEAM_SHEET_NAME}' acessada com sucesso.")
except gspread.exceptions.WorksheetNotFound:
    # 3. Se a aba não for encontrada, cria-a com um cabeçalho apropriado.
    print(f"Aba '{TEAM_SHEET_NAME}' não encontrada. Criando...")
    # Ajusta cols conforme necessário, garantindo colunas suficientes para o TEAM_HEADER
    team_worksheet = spreadsheet.add_worksheet(title=TEAM_SHEET_NAME, rows="100", cols=str(len(TEAM_HEADER)))
    # Adiciona a linha do cabeçalho
    team_worksheet.append_row(TEAM_HEADER)
    print(f"Aba '{TEAM_SHEET_NAME}' criada com cabeçalho.")

# 4. Limpa o conteúdo existente na aba "Time" (mantendo o cabeçalho).
all_values = team_worksheet.get_all_values()
if len(all_values) > 1: # Verifica se há linhas além do cabeçalho
    num_existing_rows = len(all_values)
    # Define o intervalo a ser limpo (da segunda linha até a última linha)
    range_to_clear = f'A2:{chr(ord("A") + len(TEAM_HEADER) - 1)}{num_existing_rows}'
    print(f"Limpando intervalo: {range_to_clear}")
    try:
        team_worksheet.batch_clear([range_to_clear])
        print("Conteúdo existente limpo (mantendo cabeçalho).")
    except Exception as e:
        print(f"Erro ao limpar conteúdo da planilha '{TEAM_SHEET_NAME}': {e}")


# 5. Escreve os dados preparados na aba "Time".
if team_rows_to_write:
    try:
        # Escreve as linhas começando da segunda linha (após o cabeçalho)
        team_worksheet.append_rows(team_rows_to_write, value_input_option='USER_ENTERED') # USER_ENTERED para processar fórmulas
        print(f"{len(team_rows_to_write)} linhas de dados escritas em '{TEAM_SHEET_NAME}'.")
    except Exception as e:
        print(f"Erro ao escrever dados na planilha '{TEAM_SHEET_NAME}': {e}")

    # 6. Aplica cor de fundo às células da coluna "Tipo(s)" e formata a coluna "Sprite".
    start_row_index = 2 # O número da linha no Google Sheets começa em 1, os dados começam na linha 2
    type_column_index = 1 # A coluna "Tipo(s)" é a segunda coluna (índice 1)
    sprite_column_index = 2 # A coluna "Sprite" é a terceira coluna (índice 2)

    # Prepare uma lista de formatos de célula para aplicar em uma atualização em lote para eficiência
    cell_formats = []

    for i, row_data in enumerate(team_rows_to_write):
        # Formata a coluna Sprite como IMAGEM (já feito em team_rows_to_write, mas pode verificar o formato aqui se necessário)
        # A fórmula IMAGE já está em team_rows_to_write, então só precisamos garantir que a coluna seja larga o suficiente, se necessário (passo manual ou separado)
        # Por enquanto, foque em colorir a célula do Tipo

        types_str = row_data[type_column_index] # Obtém a string de tipos dos dados da linha preparados
        pokemon_types = [t.strip().lower() for t in types_str.split(',') if t.strip()] # Divide e limpa os tipos

        # Aplica cor com base no primeiro tipo encontrado, se houver
        if pokemon_types:
            primary_type = pokemon_types[0]
            color_code = pokemon_type_colors.get(primary_type)

            if color_code:
                # O endereço da célula está na coluna do tipo (B) e no número da linha atual
                cell_address = f'{chr(ord("A") + type_column_index)}{start_row_index + i}'
                cell_formats.append({
                    "range": cell_address,
                    "format": {
                        "backgroundColor": {
                            "red": int(color_code[1:3], 16) / 255,
                            "green": int(color_code[3:5], 16) / 255,
                            "blue": int(color_code[5:7], 16) / 255
                        }
                    }
                })

    # Aplica os formatos de célula coletados em uma atualização em lote
    if cell_formats:
        try:
            team_worksheet.batch_format(cell_formats)
            print(f"Aplicada formatação de cor para {len(cell_formats)} células na coluna 'Tipo(s)'.")
        except Exception as e:
            print(f"Erro ao aplicar formatação em lote na coluna 'Tipo(s)': {e}")


    # 7. Opcional: Aplicar formatação básica ao cabeçalho.
    header_range = f'A1:{chr(ord("A") + len(TEAM_HEADER) - 1)}1'
    try:
        team_worksheet.format(header_range, {
            "backgroundColor": {
                "red": 0.85, "green": 0.85, "blue": 0.85 # Fundo cinza claro
            },
            "textFormat": {
                "bold": True
            }
        })
        print("Formatação aplicada ao cabeçalho.")
    except Exception as e:
        print(f"Erro ao formatar o cabeçalho: {e}")

else:
    print("Nenhuma linha para escrever na aba 'Time'.")

print("Escrita de dados organizados na aba 'Time' finalizada.")

Preparando para escrever dados na planilha 'Time'...
Aba 'Time' acessada com sucesso.
Limpando intervalo: A2:D7
Conteúdo existente limpo (mantendo cabeçalho).
6 linhas de dados escritas em 'Time'.
Aplicada formatação de cor para 6 células na coluna 'Tipo(s)'.
Formatação aplicada ao cabeçalho.
Escrita de dados organizados na aba 'Time' finalizada.


Cria a aba Time

In [265]:
# Esta célula orquestra os passos finais para processar a resposta do Agente de IA e escrever na aba 'Time'.

# Passo 1: Processar a resposta do Agente de IA para extrair detalhes da equipe (Nome do Pokémon e estratégia)
# Este passo já está presente nas saídas de execução das subtarefas "Processar a resposta do agente de ia"
# e "Limpa a Estratégia". O resultado está armazenado na lista suggested_team_details.
# Podemos re-executar a etapa final de limpeza por precaução.
if suggested_team_details:
    last_entry = suggested_team_details[-1]
    strategy = last_entry.get("Estratégia", "")
    if strategy.endswith("**"):
        last_entry["Estratégia"] = strategy[:-2].strip()
    print("Detalhes finais da equipe sugerida após limpeza:")
    display(suggested_team_details)
else:
    print("Nenhum detalhe de equipe sugerida encontrado para processar.")


# Passo 2: Coletar dados dos Pokémon sugeridos (tipos e URLs de sprite)
# Este passo já está presente nas saídas de execução da subtarefa "Coletar dados dos pokémon sugeridos".
# O resultado está armazenado na lista team_pokemon_data.
# Precisamos re-executar esta parte para popular team_pokemon_data com base nos detalhes limpos da equipe sugerida.

team_pokemon_data = []
if suggested_team_details:
    print("\nColetando dados detalhados para a equipe sugerida...")
    for pokemon_detail in suggested_team_details:
        pokemon_name = pokemon_detail.get("Nome do Pokemon")
        strategy = pokemon_detail.get("Estratégia", "")

        if pokemon_name:
            types, sprite_url = get_pokemon_types(pokemon_name) # Use a função existente
            if types is not None:
                pokemon_data = {
                    "Nome do Pokemon": pokemon_name,
                    "Tipos": types,
                    "Sprite URL": sprite_url,
                    "Estratégia": strategy
                }
                team_pokemon_data.append(pokemon_data)
            else:
                print(f"Não foi possível obter tipos e sprite para '{pokemon_name}'. Pulando.")
        else:
            print("Nome do Pokémon não encontrado no detalhe. Pulando.") # Esta linha estava incompleta no prompt do usuário, corrigi para a linha completa.


# Passo 3: Preparar dados para a aba 'Time'
# Este passo já está presente nas saídas de execução da subtarefa "Preparar dados para a aba 'time'".
# O resultado está armazenado na lista team_rows_to_write.
# Precisamos re-executar esta parte para popular team_rows_to_write com base em team_pokemon_data.

team_rows_to_write = []
if team_pokemon_data:
    print("\nPreparando dados para a aba 'Time'...")
    for pokemon_data in team_pokemon_data:
        pokemon_name = pokemon_data.get("Nome do Pokemon", "N/A")
        pokemon_types = pokemon_data.get("Tipos", [])
        sprite_url = pokemon_data.get("Sprite URL")
        strategy = pokemon_data.get("Estratégia", "N/A")

        types_str = ", ".join([t.capitalize() for t in pokemon_types]) if pokemon_types else "N/A"
        sprite_formula = f'=IMAGE("{sprite_url}")' if sprite_url else "N/A"

        row = [pokemon_name, types_str, sprite_formula, strategy]
        team_rows_to_write.append(row)
    print("Preparação de dados para a aba 'Time' finalizada.")
    print("\nLinhas para escrever:")
    display(team_rows_to_write)
else:
    print("Nenhum dado de equipe disponível para preparar linhas para a aba 'Time'.")


# Passo 4: Criar e formatar a aba 'Time' e escrever os dados preparados
# Esta lógica já está presente nas saídas da subtarefa "Criar e formatar a aba 'time'".
# A lógica para criar e formatar a aba não foi encapsulada em uma função, mas executada diretamente.
# Reproduziremos essa lógica aqui.


TEAM_SHEET_NAME = 'Time'
TEAM_HEADER = ["Nome", "Tipo(s)", "Sprite", "Estratégia Sugerida"]

print(f"\nEscrevendo dados e formatando planilha '{TEAM_SHEET_NAME}'...")

try:
    team_worksheet = spreadsheet.worksheet(TEAM_SHEET_NAME)
    print(f"Aba '{TEAM_SHEET_NAME}' acessada com sucesso.")
except gspread.exceptions.WorksheetNotFound:
    print(f"Aba '{TEAM_SHEET_NAME}' não encontrada. Criando...")
    team_worksheet = spreadsheet.add_worksheet(title=TEAM_SHEET_NAME, rows="100", cols=str(len(TEAM_HEADER)))
    team_worksheet.append_row(TEAM_HEADER)
    print(f"Aba '{TEAM_SHEET_NAME}' criada com cabeçalho.")

all_values = team_worksheet.get_all_values()
if len(all_values) > 1:
    num_existing_rows = len(all_values)
    range_to_clear = f'A2:{chr(ord("A") + len(TEAM_HEADER) - 1)}{num_existing_rows}'
    print(f"Limpando intervalo: {range_to_clear}")
    try:
        team_worksheet.batch_clear([range_to_clear])
        print("Conteúdo existente limpo (mantendo cabeçalho).")
    except Exception as e:
        print(f"Erro ao limpar conteúdo da planilha '{TEAM_SHEET_NAME}': {e}")

if team_rows_to_write:
    try:
        team_worksheet.append_rows(team_rows_to_write, value_input_option='USER_ENTERED')
        print(f"{len(team_rows_to_write)} linhas de dados escritas em '{TEAM_SHEET_NAME}'.")
    except Exception as e:
        print(f"Erro ao escrever dados na planilha '{TEAM_SHEET_NAME}': {e}")

    start_row_index = 2
    type_column_index = 1
    sprite_column_index = 2

    cell_formats = []

    for i, row_data in enumerate(team_rows_to_write):
        types_str = row_data[type_column_index]
        pokemon_types = [t.strip().lower() for t in types_str.split(',') if t.strip()]

        if pokemon_types:
            primary_type = pokemon_types[0]
            color_code = pokemon_type_colors.get(primary_type)

            if color_code:
                cell_address = f'{chr(ord("A") + type_column_index)}{start_row_index + i}'
                cell_formats.append({
                    "range": cell_address,
                    "format": {
                        "backgroundColor": {
                            "red": int(color_code[1:3], 16) / 255,
                            "green": int(color_code[3:5], 16) / 255,
                            "blue": int(color_code[5:7], 16) / 255
                        }
                    }
                })

    if cell_formats:
        try:
            team_worksheet.batch_format(cell_formats)
            print(f"Aplicada formatação de cor para {len(cell_formats)} células na coluna 'Tipo(s)'.")
        except Exception as e:
            print(f"Erro ao aplicar formatação em lote na coluna 'Tipo(s)': {e}")

    header_range = f'A1:{chr(ord("A") + len(TEAM_HEADER) - 1)}1'
    try:
        team_worksheet.format(header_range, {
            "backgroundColor": {
                "red": 0.85, "green": 0.85, "blue": 0.85
            },
            "textFormat": {
                "bold": True
            }
        })
        print("Formatação aplicada ao cabeçalho.")
    except Exception as e:
        print(f"Erro ao formatar o cabeçalho: {e}")

else:
    print("Nenhuma linha para escrever na aba 'Time'.")

print("Escrita de dados organizados na aba 'Time' finalizada.")

Detalhes finais da equipe sugerida após limpeza:


[{'Nome do Pokemon': 'Torracat',
  'Estratégia': 'Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.'},
 {'Nome do Pokemon': 'Dragalge',
  'Estratégia': 'Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.'},
 {'Nome do Pokemon': 'Palpitoad',
  'Estratégia': 'Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.'},
 {'Nome do Pokemon': 'Sinistea',
  'Estratégia': 'O tipo fantasma é imune a ataques normais e luta, oferecendo uma camada importante de defesa.  É efetivo contra fantasmas, psicicos e outros tipos, e cobre as fraquezas dos outros.'},
 {'Nome do Pokemon': 'Primeape',
  'Estratégia': 'Oferece cobertura para tipos de rocha, aço, gelo e normal que os outros Pokémon do time não abordam tão bem.'},
 {'Nome do Pokemon': 'Joltik',
  'Estratégia'


Coletando dados detalhados para a equipe sugerida...
Buscando dados para 'torracat' na PokeAPI...
Tipos encontrados: ['fire']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png
Buscando dados para 'dragalge' na PokeAPI...
Tipos encontrados: ['poison', 'dragon']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/691.png
Buscando dados para 'palpitoad' na PokeAPI...
Tipos encontrados: ['water', 'ground']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/536.png
Buscando dados para 'sinistea' na PokeAPI...
Tipos encontrados: ['ghost']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/854.png
Buscando dados para 'primeape' na PokeAPI...
Tipos encontrados: ['fighting']
URL da sprite encontrada: https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/57.png
Buscando dados par

[['Torracat',
  'Fire',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/726.png")',
  'Fogo é um tipo ofensivo forte,  eficaz contra plantas, gelo, inseto e aço. É um bom atacante inicial.'],
 ['Dragalge',
  'Poison, Dragon',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/691.png")',
  'Cobertura excelente contra Dragão, que é uma fraqueza de Torracat.  O tipo veneno também é vantajoso contra lutadores e plantas.'],
 ['Palpitoad',
  'Water, Ground',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/536.png")',
  'Uma excelente escolha para cobrir as fraquezas de Torracat e Dragalge, particularmente contra Fogo e Elétrico.  A combinação de Água/Terra oferece boa defesa e ataque.'],
 ['Sinistea',
  'Ghost',
  '=IMAGE("https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/854.png")',
  'O tipo fantasma é imune a ataques normais e luta, oferecendo uma camada importa


Escrevendo dados e formatando planilha 'Time'...
Aba 'Time' acessada com sucesso.
Limpando intervalo: A2:D7
Conteúdo existente limpo (mantendo cabeçalho).
6 linhas de dados escritas em 'Time'.
Aplicada formatação de cor para 6 células na coluna 'Tipo(s)'.
Formatação aplicada ao cabeçalho.
Escrita de dados organizados na aba 'Time' finalizada.
